solve
inspired by https://github.com/AnnZhouCcc/topobench/blob/master/lpmaker/graphs/Graph.java#L1382

In [29]:
# set parameters
graphname = "dring"
routingname = "su2"
pwname = ""
numsw = 80
numport = 64
numserver = 2988
numfaillink = 0
numlink = 2132
fseed = 0
homedir = "/home/annzhou"

c=64
s=4
multstr="1_0_0"
tseed = 0
flowfile = f"{homedir}/DRing/src/emp/datacentre/flowfiles/c2s_{c}_{s}_{multstr}_{tseed}"

if graphname=="dring" or graphname=="rrg":
    netpathfile = f"{homedir}/DRing/src/emp/datacentre/netpathfiles/netpath_{routingname}_{graphname}.txt"
else:
    print("ERROR: leafspine does not need to compute path weight")

if graphname=="dring":
    graphfile = f"{homedir}/DRing/src/emp/datacentre/graphfiles/ring_supergraph/double_ring/instance1_{numsw}_{numport}.edgelist"
elif graphname=="rrg":
    graphfile = f"{homedir}/DRing/src/emp/datacentre/graphfiles/ring_supergraph/rrg/instance1_{numsw}_{numport}.edgelist"
else:
    print("ERROR: leafspine does not need to compute path weight")

linkfailurefile = f"{homedir}/DRing/src/emp/datacentre/linkfailurefiles/{graphname}_{numlink}_{numfaillink}_{fseed}"

serverfile = f"{homedir}/DRing/src/emp/datacentre/serverfiles/{graphname}_{numserver}_{numsw}_{numport}"

varfile = f"{homedir}/DRing/src/emp/datacentre/makepathweightfiles/varfile"
modelfile = f"{homedir}/DRing/src/emp/datacentre/makepathweightfiles/model.lp"

In [22]:
# dirprefix = f"{homedir}/DRing/src/emp/datacentre"
# netpathfile = f"{dirprefix}/netpathfiles/node3.txt"
# graphfile = f"{dirprefix}/graphfiles/node3.edgelist"
# serverfile = f"{dirprefix}/serverfiles/node3"
# flowfile = f"{dirprefix}/flowfiles/node3"

In [30]:
# read netpathfile
netpath = list()
for i in range(numsw):
    netpath.append(list())
    for j in range(numsw):
        netpath[i].append(list())
with open(netpathfile,'r') as f:
    lines = f.readlines()
    # produce
    fromsw = 0
    tosw = 0
    for line in lines:
        if "->" not in line:
            tokens = line.split()
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
        else:
            path = [fromsw]
            tokens = line.split()
            for token in tokens:
                hops = token.split("->")
                path.append(int(hops[1]))
            netpath[fromsw][tosw].append(path)

    # check
    for line in lines:
        if "->" not in line:
            tokens = line.split()
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            numpaths = int(tokens[2])
            if len(netpath[fromsw][tosw])!=numpaths:
                print(f"ERROR: netpath is wrong, fromsw={fromsw}, tosw={tosw}, numpaths from file={numpaths}, numpaths from array={len(netpath[fromsw][tosw])}")

# read graphfile
link = list()
for i in range(numsw):
    link.append(list())
    for j in range(numsw):
        link[i].append(0)
with open(graphfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split("->")
        fromsw = int(tokens[0])
        tosw = int(tokens[1])
        link[fromsw][tosw] = 1
        link[tosw][fromsw] = 1

# read linkfailurefile (if needed)
if numfaillink > 0:
    with open(linkfailurefile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split()
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            if link[fromsw][tosw] != 1:
                print(f"ERROR: should have a link from {fromsw} to {tosw} but not")
            else:
                link[fromsw][tosw] /= 2

# read serverfile
serverdict = dict()
with open(serverfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        serverdict[int(tokens[0])] = int(tokens[1])

# read flowfile
traffic = list()
for i in range(numsw):
    traffic.append(list())
    for j in range(numsw):
        traffic[i].append(0)
with open(flowfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(",")
        fromsvr = int(tokens[0])
        tosvr = int(tokens[1])
        if fromsvr>=numserver or tosvr>=numserver: continue
        fromsw = serverdict[fromsvr]
        tosw = serverdict[tosvr]
        if fromsw == tosw: continue
        flowbytes = int(tokens[2])
        traffic[fromsw][tosw] += flowbytes

In [33]:
# precompute
flowsvialink = list()
for i in range(numsw):
    flowsvialink.append(list())
    for j in range(numsw):
        flowsvialink[i].append(list())
for fromsw in range(numsw):
    for tosw in range(numsw):
        if traffic[fromsw][tosw] > 0:
            for pid,path in enumerate(netpath[fromsw][tosw]):
                fidpidstr = f"{fromsw},{tosw},{pid}"
                prevhop = fromsw
                for hop in path[1:]:
                    flowsvialink[prevhop][hop].append(fidpidstr)
                    prevhop = hop

for fromsw in range(numsw):
    for tosw in range(numsw):
        traffic[fromsw][tosw] /= 100000

In [34]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
model = gp.Model("mcf")

# Add variables
maxpid = 0
for fromsw in range(numsw):
    for tosw in range(numsw):
        maxpid = max(maxpid,len(netpath[fromsw][tosw]))
vararr = list()
for fromsw in range(numsw):
    vararr.append(list())
    for tosw in range(numsw):
        vararr[fromsw].append(list())
        for pid in range(maxpid):
            vararr[fromsw][tosw].append(None)
for fromsw in range(numsw):
    for tosw in range(numsw):
        if traffic[fromsw][tosw] > 0:
            for pid in range(len(netpath[fromsw][tosw])):
                var = model.addVar(name=f"{fromsw}_{tosw}_{pid}")
                vararr[fromsw][tosw][pid] = var
k = model.addVar(name="k")

# Set objective function
model.setObjective(k, GRB.MAXIMIZE)

# Add constraints
# Constraint 0: for each fid: sum(pid) >= k * traffic[fid.from][fid.to]
for fromsw in range(numsw):
    for tosw in range(numsw):
        if traffic[fromsw][tosw] > 0:
            varlist = list()
            for pid in range(len(netpath[fromsw][tosw])):
                varlist.append(vararr[fromsw][tosw][pid])
            model.addConstr(sum(varlist)>=k*traffic[fromsw][tosw],f"c0_{fromsw}_{tosw}")

# Constraint 1: for each link: sum(fid_pid) <= link[link.from][link.to]
for linkfrom in range(numsw):
    for linkto in range(numsw):
        flowstrlist = flowsvialink[linkfrom][linkto]
        if len(flowstrlist) > 0:
            varlist = list()
            for flowstr in flowstrlist:
                tokens = flowstr.split(',')
                flowfrom = int(tokens[0])
                flowto = int(tokens[1])
                pid = int(tokens[2])
                varlist.append(vararr[flowfrom][flowto][pid])
            model.addConstr(sum(varlist)<=link[linkfrom][linkto],f"c1_{linkfrom}_{linkto}")

# Optimize model
model.optimize()
model.write(modelfile)

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {model.objVal}")
    with open(varfile,'w') as f:
        f.write(f"{k.varName},{k.x}\n")
        for fromsw in range(numsw):
            for tosw in range(numsw):
                if traffic[fromsw][tosw] > 0:
                    for pid in range(len(netpath[fromsw][tosw])):
                        var = vararr[fromsw][tosw][pid]
                        f.write(f"{var.varName},{var.x}\n")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 40 logical processors, using up to 20 threads

Optimize a model with 2582 rows, 24429 columns and 92464 nonzeros
Model fingerprint: 0xbc11bd95
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 48 rows and 0 columns
Presolve time: 0.07s
Presolved: 2534 rows, 24429 columns, 92416 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.03s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 6.616e+04
 Factor NZ  : 2.186e+05 (roughly 13 MB of memory)
 Factor Ops : 3.604e+07 (less than 1 second per iteration)
 Threads    : 18

Barrier performed 0 iterations in 0.18 seconds (0.09 work units)
Barrier solv

In [ ]:
# quantize
# inspired by 